In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,remove_punction,normalize,get_year,get_countries
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache


In [12]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [13]:
df_ipcc['freq']=1

In [14]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [17]:
df_ipcc_group=df_ipcc.groupby(by=['doi','year'], dropna=True).agg({'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [18]:
df_ipcc_group[df_ipcc_group.freq>1]

,doi,year,ipcc,countries,freq
4,10.1001/jama.2020.15398,2020,"[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7,...",[US],3
17,10.1002/(sici)1520-6505(1998)7:3<81::aid-evan3...,1998,"[{'name': 'wg2_chap_01', 'wg': '2', 'chap': 1,...",[US],2
27,10.1002/2013ef000194,2014,"[{'name': 'wg2_chap_11', 'wg': '2', 'chap': 11...",[AU],2
28,10.1002/2013ef000199,2014,"[{'name': 'wg3_chap_03', 'wg': '3', 'chap': 3,...","[US, DE]",2
32,10.1002/2013gb004664,2014,"[{'name': 'wg2_chap_12', 'wg': '2', 'chap': 12...","[GB, NL, BR]",2
...,...,...,...,...,...
57229,10.7930/nca4.2018.ch26,2018,"[{'name': 'wg1_chap_12', 'wg': '1', 'chap': 12...",[None],3
57232,10.7930/nca4.2018.ch3,2018,"[{'name': 'wg2_chap_04', 'wg': '2', 'chap': 4,...",[None],2
57239,10.9734/ajaees/2013/1856,2013,"[{'name': 'wg2_cross_chap_7', 'wg': '2_cross',...",[NG],3
57240,10.9734/ajfar/2018/v2i330011,2019,"[{'name': 'wg2_chap_15', 'wg': '2', 'chap': 15...",[FJ],2


In [22]:
df_ipcc_group.ipcc[4]

[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7, 'freq_ipcc': 1},
 {'name': 'wg2_chap_04', 'wg': '2', 'chap': 4, 'freq_ipcc': 2},
 {'name': 'wg2_cross_chap_2', 'wg': '2_cross', 'chap': 2, 'freq_ipcc': 2}]

In [8]:
df_ipcc_group.doi[4]

'10.1001/jama.2020.15398'

In [ ]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [ ]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [ ]:
df_ipbes_group=df_ipbes.groupby(by=['doi','year'], dropna=True).agg({'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipbes_group=df_ipbes_group.reset_index()

In [ ]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [ ]:
df_ipbes['year']=df_ipbes.loc[:,'date'].apply(lambda x: get_year(str(x)) if pd.isna(x)==False else None)

In [ ]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes

In [ ]:
update_bso_publications()

In [ ]:
bso=pd.DataFrame(get_bso_publications())

In [ ]:
df_bso=bso.transpose().reset_index()

In [ ]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [ ]:
df_bso

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left')

In [ ]:
df_ipcc_ipbes_bso['countries']=df_ipcc_ipbes_bso.apply(lambda row: get_countries(row),axis=1)

In [ ]:
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso[['doi', 'ipcc', 'ipbes', 'countries', 'rors', 'bso_local_affiliations']]

In [ ]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [ ]:
df_iso=df_iso.rename(columns={'code':'countries'})

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_iso, on='countries', how='left')
del df_ipcc_ipbes_bso['countries']
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso.rename(columns={'name':'countries'})

In [ ]:
df_brevet_doi=pd.DataFrame('brevet_doi.json')

In [ ]:
df_brevet_doi

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [ ]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

In [ ]:
df_cache

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [ ]:
dict_doi_bso=[]

In [ ]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [ ]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [ ]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', lines=True)

In [ ]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [ ]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

In [ ]:
current_dict = dd.to_dict(orient='records')

In [ ]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [ ]:
df_elastic_input=pd.DataFrame(current_dict)

In [ ]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [ ]:
del df_elastic_input['openalex_abstract_inverted_index']

In [ ]:
len(df_elastic_input)

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [ ]:
#df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'